In [1]:
import os
import time
import itertools 

In [2]:
def load_data(filename):
    file = open(filename, 'r')
    
    # list containing baskets as list of lists [[1st basket],[2nd basket]]
    baskets = []
    for line in file: 
        basket = line.split(' ')
        basket.remove('\n')
        basket = list(map(int, basket))
        baskets.append(basket)
        
    return baskets

In [3]:
def load_data_slide(filename):
    file = open(filename, 'r')
    
    # list containing baskets as list of lists [[1st basket],[2nd basket]]
    baskets = []
    for line in file: 
        basket = line.split(' ')
        basket = list(map(int, basket))
        baskets.append(basket)
        
    return baskets

In [4]:
# generate C1
def generate_candidates(baskets):

    candidates = {}

    for basket in baskets:
        for items in basket:
            if candidates.get(items) == None:
                    candidates[items] = 1
            else:
                candidates[items] += 1


    return candidates

In [5]:
# generate C2 -> NAIVE SOLUTION
def generate_candidates_2(frequent_items, baskets):

    items = sorted(frequent_items.keys())
    couples = list(itertools.combinations(items, 2))
    # print("couples: ", couples)

    C2 = {}

    for basket in baskets:
        for couple in couples:
            if all(item in basket for item in couple):
                if C2.get(couple) == None:
                    C2[couple] = 1
                else: 
                    C2[couple] += 1
                    
    return C2

In [6]:
# ideale per C2
def generate_candidates_Ck(frequent_items, baskets, k_tuple):

    items = sorted(frequent_items.keys())
    Ck = {}
    
    for basket in baskets:
        basket = set(basket)
        common_elements = basket.intersection(set(items))
        itemsets = list(itertools.combinations(sorted(common_elements), k_tuple))
        
        for itemset in itemsets:
            if Ck.get(itemset) == None:
                Ck[itemset] = 1
            else: 
                Ck[itemset] += 1
    return Ck

In [7]:
# per C3 ma senza eliminare da C3 le coppie del cazzo 
def generate_candidates_Ckn(frequent_items, baskets, k_tuple):

    #items = set(itertools.chain(*frequent_items.keys()))
    items = [item for sublist in frequent_items.keys() for item in sublist]

    Ck = {}
    
    for basket in baskets:
        basket = set(basket)
        common_elements = basket.intersection(set(items))
        itemsets = list(itertools.combinations(sorted(common_elements), k_tuple))
        
        for itemset in itemsets:
            if Ck.get(itemset) == None:
                Ck[itemset] = 1
            else: 
                Ck[itemset] += 1
    return Ck

In [28]:
# per C3 eliminando le coppie inutili, 1o metodo
def generate_candidates_Ckn_mod(frequent_items, baskets, k_tuple):

    #items = set(itertools.chain(*frequent_items.keys()))
    items = [item for sublist in frequent_items.keys() for item in sublist]
    items = sorted(list(set(items)))
    #print("ciaone", items)
    lk_1_items = list(frequent_items.keys())
    #print("daje", lk_1_items)
    
    Ck = {}
    
    for basket in baskets:
        basket = set(basket)
        common_elements = basket.intersection(set(items))
        itemsets = list(itertools.combinations(sorted(common_elements), k_tuple))
                
        for itemset in itemsets:
            combinations = list(itertools.combinations(itemset, k_tuple-1))            
            if all(item in lk_1_items for item in combinations):
                if Ck.get(itemset) is None:
                    Ck[itemset] = 1
                else: 
                    Ck[itemset] += 1
    return Ck

In [27]:
# per C3 eliminando le coppie inutili, 2o metodo
def generate_candidates_Ckn_mod_adv(frequent_items, baskets, k_tuple):

    #items = set(itertools.chain(*frequent_items.keys()))
    items = [item for sublist in frequent_items.keys() for item in sublist]
    lk_1_items = list(frequent_items.keys())
    
    Ck = {}
    
    for basket in baskets:
        basket = set(basket)
        common_elements = basket.intersection(set(items))
        itemsets = list(itertools.combinations(sorted(common_elements), k_tuple))
                
        for itemset in itemsets:
            combinations = list(itertools.combinations(itemset, k_tuple-1))
            combinations = set(combinations)
            check_combinations = combinations.intersection(set(lk_1_items))
            
            if (check_combinations == combinations):
                if Ck.get(itemset) is None:
                    Ck[itemset] = 1
                else: 
                    Ck[itemset] += 1
    return Ck

In [25]:
# per C3 eliminando le coppie inutili, 3o metodo
def generate_candidates_Ckn_mod_adv_again(frequent_items, baskets, k_tuple):

    #items = set(itertools.chain(*frequent_items.keys()))
    items = [item for sublist in frequent_items.keys() for item in sublist]
    lk_1_items = list(frequent_items.keys())
    
    Ck = {}
    
    for basket in baskets:
        basket = set(basket)
        common_elements = basket.intersection(set(items))
        itemsets = list(itertools.combinations(sorted(common_elements), k_tuple))
                
        for itemset in itemsets:
            combinations = list(itertools.combinations(itemset, 2))
            print("itemset", itemset)
            print("combinations", combinations)
            print("lk_1_items", lk_1_items)
            if (combinations in lk_1_items):
                if Ck.get(itemset) is None:
                    Ck[itemset] = 1
                else: 
                    Ck[itemset] += 1
    return Ck

In [13]:
# input L2, L1 -> generate C3
def generate_candidates_3(frequent_items, singletons_items, baskets):
    
    C3 = {}
    
    singletons = sorted(list(singletons_items.keys()))
    items = list(frequent_items.keys())
    
    #print(items)
    #print(singletons)
    
    # ciclo su ogni item di L2 e su ogni item di L1, se l'elemento di L1 non è in L2 allora
    # creo la tripletta. Dopo controllo che ogni combinazione (subset) di 2 elementi della tripletta
    # sia presente in L2 per rispettare la regola, se si piazzo in C3 con count = 0.
    # funzione del count da fare a sè così da diminuire anche il carico di C2
    candidate = []
    for item in items:
        for singleton in singletons:
            if singleton not in item:
                candidate = list(item)
                candidate.append(singleton)
                #print(candidate)
                cand_comb = list(itertools.combinations(sorted(candidate),2))
                for comb in cand_comb:
                    if all(item in items for item in cand_comb):
                        C3[tuple(candidate)] = 0
                candidate.clear()
                
    end = time.time()
    #print(C3)
    return C3
            

In [14]:
def filter_candidates(candidates, support):

    frequent_items = {}
    for item, occurences in list(candidates.items()):
        if (occurences >= support):
            frequent_items[item] = occurences

    return frequent_items

In [35]:
starting_point = time.time()
filename = '../data/T10I4D100K.dat'
baskets = load_data(filename)

start = time.time()
C1 = generate_candidates(baskets)
end = time.time()
print("Time C1: ", end-start)

L1 = filter_candidates(C1, 500)
print(len(L1.keys()))

start = time.time()
Ck = generate_candidates_Ck(L1, baskets, 2)
end = time.time()
print("Time Ck=2 ", end-start)

L2 = filter_candidates(Ck, 500)
print(len(L2.keys()))

# C3 metodo montesi 
start = time.time()
C3 = generate_candidates_Ckn(L2, baskets, 3)
end = time.time()
print("Time C3_montesi ", end-start)

# L3 
L3 = filter_candidates(C3, 500)
print("L3 len: ", len(L3.keys()))

# C4 metodo montesi
start = time.time()
C4 = generate_candidates_Ckn(L3, baskets, 4)
end = time.time()
print("Time C4_montesi ", end-start)

# L4
L4 = filter_candidates(C4, 500)
print("L4 len: ", len(L4.keys()))

# C5 metodo montesi
start = time.time()
C5 = generate_candidates_Ckn(L4, baskets, 5)
end = time.time()
print("Time C5_montesi ", end-start)

# L5
L5 = filter_candidates(C5, 500)
print("L5 len: ", len(L5.keys()))

# C5 metodo montesi
start = time.time()
C6 = generate_candidates_Ckn(L5, baskets, 6)
end = time.time()
print("Time C6_montesi ", end-start)

# L6
L6 = filter_candidates(C6, 500)
print("L6 len: ", len(L6.keys()))

# C7
start = time.time()
C7 = generate_candidates_Ckn(L6, baskets, 7)
end = time.time()
print("Time C7_montesi ", end-start)

# L7
L7 = filter_candidates(C7, 500)
print("L7 len: ", len(L7.keys()))


# # C3 mod 
# start = time.time()
# C3_mod = generate_candidates_Ckn_mod(L2, baskets, 3)
# end = time.time()
# print("Time C3_mod ", end-start)

# # C3 mod adv
# start = time.time()
# C3_mod_adv = generate_candidates_Ckn_mod_adv(L2, baskets, 3)
# end = time.time()
# print("Time C3_mod_adv ", end-start)

# # L3 post mod adv
# L3_mod_adv = filter_candidates(C3_mod_adv, 800)
# print("L3_mod_adv len: ", len(L3_mod_adv.keys()))

# # C4 mod 
# start = time.time()
# C4_mod = generate_candidates_Ckn_mod(L3, baskets, 4)
# end = time.time()
# print("Time C4_mod ", end-start)

# # C4 mod adv
# start = time.time()
# C4_mod_adv = generate_candidates_Ckn_mod_adv(L3, baskets, 4)
# end = time.time()
# print("Time C3_mod_adv ", end-start)

# # L4 mod adv
# L4_mod_adv = filter_candidates(C4_mod_adv, 800)
# print("L4_mod_adv len: ", len(L4_mod_adv.keys()))


end_point = time.time()
print("Total time: ", end_point-starting_point)

Time C1:  0.24074888229370117
569
Time Ck=2  4.306578874588013
342
Time C3_montesi  3.9291961193084717
L3 len:  110
Time C4_montesi  0.9339568614959717
L4 len:  43
Time C5_montesi  0.4657628536224365
L5 len:  9
Time C6_montesi  0.25752806663513184
L6 len:  0
Time C7_montesi  0.15405726432800293
L7 len:  0
Total time:  11.046463966369629


In [26]:
filename = '../data/slide.dat'
baskets = load_data_slide(filename)

start = time.time()
C1 = generate_candidates(baskets)
end = time.time()
print("Time C1: ", end-start)

start = time.time()
L1 = filter_candidates(C1, 2)
end = time.time()
print("Time L1: ", end-start)

start = time.time()
Ck = generate_candidates_Ck(L1, baskets, 2)
end = time.time()
print("Time Ck=2 ", end-start)
print(Ck)

start = time.time()
L2 = filter_candidates(Ck, 2)
end = time.time()
print("Time L2 ", end-start)
print(L2)

start = time.time()
C3_1 = generate_candidates_Ckn(L2, baskets, 3)
end = time.time()
print("Time C3_1 ", end-start)
print(C3_1)

start = time.time()
C3_2 = generate_candidates_Ckn_mod_adv(L2, baskets, 3)
end = time.time()
print("Time C3_2 ", end-start)
print(C3_2)

start = time.time()
C3_3 = generate_candidates_Ckn_mod_adv_again(L2, baskets, 3)
end = time.time()
print("Time C3_3 ", end-start)
print(C3_3)

Time C1:  7.104873657226562e-05
Time L1:  8.296966552734375e-05
Time Ck=2  8.893013000488281e-05
{(1, 3): 2, (2, 3): 2, (2, 5): 3, (3, 5): 2, (1, 2): 1, (1, 5): 1}
Time L2  6.103515625e-05
{(1, 3): 2, (2, 3): 2, (2, 5): 3, (3, 5): 2}
Time C3_1  8.177757263183594e-05
{(2, 3, 5): 2, (1, 2, 3): 1, (1, 2, 5): 1, (1, 3, 5): 1}
Time C3_2  0.00010895729064941406
{(2, 3, 5): 2}
itemset (2, 3, 5)
combinations [(2, 3), (2, 5), (3, 5)]
lk_1_items [(1, 3), (2, 3), (2, 5), (3, 5)]
itemset (1, 2, 3)
combinations [(1, 2), (1, 3), (2, 3)]
lk_1_items [(1, 3), (2, 3), (2, 5), (3, 5)]
itemset (1, 2, 5)
combinations [(1, 2), (1, 5), (2, 5)]
lk_1_items [(1, 3), (2, 3), (2, 5), (3, 5)]
itemset (1, 3, 5)
combinations [(1, 3), (1, 5), (3, 5)]
lk_1_items [(1, 3), (2, 3), (2, 5), (3, 5)]
itemset (2, 3, 5)
combinations [(2, 3), (2, 5), (3, 5)]
lk_1_items [(1, 3), (2, 3), (2, 5), (3, 5)]
Time C3_3  0.0013439655303955078
{}


In [ ]:
size = [0]
size.append(len(L1.keys()))
size.append(len(L2.keys()))
size.append(len(L3.keys()))

candidat = [0]
candidat.append(len(C1.keys()))
candidat.append(len(C2.keys()))
candidat.append(len(C3_mod.keys()))

stage = [0,1,2,3]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.array(stage), np.array(size))
plt.ylabel("frequent itemsets")
plt.xlabel("stage")
plt.grid()
plt.show()

In [ ]:
plt.plot(np.array(stage), np.array(candidat))
plt.ylabel("candidates item")
plt.xlabel("stage")
plt.grid()
plt.show()

### TODO fare funziona unica (con 4 stage o far decidere utente o finchè non arriva a zero) + Parser
### TODO spostare fuori dal notebook (?) -> A me piace così
### TODO grafico del tempo
### TODO report